# Airline Data Analysis: SQL vs. Pandas

This notebook demonstrates advanced data analysis skills using both SQL and pandas on airline datasets. It showcases my ability to:
- Write complex SQL queries for data extraction and aggregation
- Translate SQL logic into efficient pandas code
- Validate results across different technologies
- Communicate analytical processes and insights clearly

# Data Loading and Setup

Load the datasets into pandas DataFrames and a temporary SQLite database for dual analysis

In [ ]:
import pandas as pd
import sqlite3
import os
import numpy as np
import tempfile, os.path
import matplotlib.pyplot as plt

In [ ]:
dbfile = os.path.join(tempfile.mkdtemp(), "flight_data")
print(dbfile)
conn = sqlite3.connect(dbfile)

/tmp/tmpdclaximz/flight_data


In [ ]:
# Load CSV 
airlines = pd.read_csv("airlines.csv")
airports = pd.read_csv("airports.csv")
flights = pd.read_csv("flights.csv")
planes = pd.read_csv("planes.csv")
weather = pd.read_csv("weather.csv")

# LOad SQLite 
airlines.to_sql("airlines", conn, index=False)
airports.to_sql("airports", conn, index=False)
flights.to_sql("flights", conn, index=False)
planes.to_sql("planes", conn, index=False)
weather.to_sql("weather", conn, index=False)

# Unique Engine Types

Description: Extract all unique engine types present in the fleet from the planes dataset.

Business Question: What different engine types are represented in our aircraft fleet?

In [ ]:
Engine = pd.read_sql_query("""
    SELECT DISTINCT engine 
    FROM planes
""", conn)
Engine_pd = (
    pd.DataFrame(planes.engine.unique(), columns=['engine']))
pd.testing.assert_frame_equal(Engine, Engine_pd) 

# Unique Combinations of Plane Type and Engine

Description: Identify each unique combination of aircraft type and engine configuration.

Business Question: Which combinations of aircraft type and engine are in use across our fleet?

In [ ]:
Engine_type = pd.read_sql_query("""
    SELECT DISTINCT type, engine 
    FROM planes
""", conn)
Engine_type_pd = (
    planes[['type', 'engine']].drop_duplicates().reset_index(drop=True)
)
pd.testing.assert_frame_equal(Engine_type, Engine_type_pd) 

# Count Planes by Engine Type

Description: Group planes by engine type and count how many planes fall into each category.

Business Question: How many planes do we operate for each engine type?

In [ ]:
Plane_Engine = pd.read_sql_query("""
    SELECT COUNT(*), engine 
    FROM planes 
    GROUP BY engine
""", conn)
Plane_Engine_pd = (
    planes.groupby('engine')
    .size()
    .reset_index(name='COUNT(*)')
    [['COUNT(*)', 'engine']] 
)
pd.testing.assert_frame_equal(Plane_Engine, Plane_Engine_pd) 

# Count Planes by Engine and Type

Description: Group planes by both engine type and aircraft type, then count the number in each group.

Business Question: What is the distribution of planes when considering both engine type and aircraft type?

In [ ]:
Plane_engine_type = pd.read_sql_query("""
    SELECT COUNT(*), engine, type 
    FROM planes 
    GROUP BY engine, type
""", conn)
Plane_engine_type_pd = (
    planes
    .groupby(['engine', 'type'])
    .size()
    .reset_index(name='COUNT(*)')
    [['COUNT(*)', 'engine', 'type']]
)
pd.testing.assert_frame_equal(Plane_engine_type, Plane_engine_type_pd) 

# Plane Age Statistics by Engine and Manufacturer

Description: For each engine and manufacturer combination, calculate the minimum, average, and maximum year of manufacture.

Business Question: What is the age range and average age of planes by engine type and manufacturer?

In [ ]:
Manufacture_year = pd.read_sql_query("""
    SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer 
    FROM planes 
    GROUP BY engine, manufacturer
""", conn)
Manufacture_year_pd = (
   planes.groupby(['engine', 'manufacturer'])  
    .agg(min_year=('year', 'min'),             
         avg_year=('year', 'mean'),          
         max_year=('year', 'max'))            
    .reset_index()                            
)
Manufacture_year_pd = Manufacture_year_pd.rename(columns={
    'min_year': 'MIN(year)',
    'avg_year': 'AVG(year)',
    'max_year': 'MAX(year)'
})

Manufacture_year_pd = Manufacture_year_pd[['MIN(year)', 'AVG(year)', 'MAX(year)', 'engine', 'manufacturer']]

Manufacture_year_pd_sorted = Manufacture_year_pd.sort_values(by=['engine', 'manufacturer']).reset_index(drop=True)
Manufacture_year_sorted = Manufacture_year.sort_values(by=['engine', 'manufacturer']).reset_index(drop=True)

pd.testing.assert_frame_equal(Manufacture_year, Manufacture_year_pd)

# List Planes with Known Speed

Description: Filter and list all planes that have a recorded (non-null) speed value.

Business Question: Which planes in our fleet have documented speed specifications?

In [ ]:
Plane_speeds= pd.read_sql_query("""
    SELECT * 
    FROM planes
    WHERE speed IS NOT NULL
""", conn)
Plane_speeds_pd = ( planes[planes['speed'].notna()].reset_index(drop=True)
    
)
pd.testing.assert_frame_equal(Plane_speeds, Plane_speeds_pd) 

# Find Recent, Medium-to-Large Planes

Description: Select tail numbers of planes with 150–210 seats manufactured from 2011 onwards.

Business Question: Which recent planes in our fleet are suitable for medium- to large-capacity routes?

In [ ]:
Medium_large_planes = pd.read_sql_query("""
    SELECT tailnum 
    FROM planes
    WHERE seats BETWEEN 150 AND 210 AND year >= 2011
""", conn)
Medium_large_planes_pd = (
    planes[(planes['seats'].between(150, 210)) & (planes['year'] >= 2011)][['tailnum']].reset_index(drop=True)
)
pd.testing.assert_frame_equal(Medium_large_planes, Medium_large_planes_pd) 

# Large Planes by Major Manufacturers

Description: List tail numbers, manufacturers, and seat counts for planes from Boeing, Airbus, or Embraer with more than 390 seats.

Business Question: Which high-capacity planes from major manufacturers are available in the fleet?

In [ ]:
Large_manufacturer = pd.read_sql_query("""
    SELECT tailnum, manufacturer, seats 
    FROM planes
    WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390
""", conn)
Large_manufacturer_pd = (
    planes[(planes['manufacturer'].isin(['BOEING','AIRBUS','EMBRAER'])) & 
            (planes['seats']> 390)]
    [['tailnum','manufacturer','seats']].reset_index(drop=True)
)
pd.testing.assert_frame_equal(Large_manufacturer, Large_manufacturer_pd) 

# Unique Year/Seat Combinations (Year Ascending, Seats Descending)

Description: Identify unique combinations of year and seats for planes manufactured since 2012, sorted by year (ascending) and seats (descending).

Business Question: What are the unique year and seat configurations among newer planes, and how do they trend over time?

In [ ]:
YearAsc_SeatsDesc = pd.read_sql_query("""
    SELECT DISTINCT year, seats FROM planes
    WHERE year >= 2012 
    ORDER BY year ASC, seats DESC
""", conn)
YearAsc_SeatsDesc_pd = (
    planes.loc[planes['year'] >= 2012, ['year', 'seats']]
    .drop_duplicates(subset=['year', 'seats']) 
    .sort_values(by=['year', 'seats'], ascending=[True, False])
    .reset_index(drop=True)
)
pd.testing.assert_frame_equal(YearAsc_SeatsDesc, YearAsc_SeatsDesc_pd)

# Count of Large Planes by Manufacturer

Description: Count the number of planes with more than 200 seats, grouped by manufacturer.

Business Question: Which manufacturers supply the most large-capacity planes in our fleet?

In [ ]:
Seats_manufacturer = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) FROM planes
    WHERE seats > 200 GROUP BY manufacturer
""", conn)
Seats_manufacturer_pd = (
    planes.loc[planes['seats'] > 200]
    .groupby('manufacturer')
    .size()
    .reset_index(name='COUNT(*)')
   
)
pd.testing.assert_frame_equal(Seats_manufacturer, Seats_manufacturer_pd) 

# Manufacturers with More Than 10 Planes

Description: List manufacturers that have more than 10 planes in the fleet.

Business Question: Which manufacturers are the most prevalent in our fleet?

In [ ]:
Manufacturer_more_than_ten_planes  = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) 
    FROM planes
    GROUP BY manufacturer HAVING COUNT(*) > 10
""", conn)
Manufacturer_more_than_ten_planes_pd = (
   planes.groupby('manufacturer')             
    .size()                                    
    .reset_index(name='COUNT(*)')   
    .loc[lambda df: df['COUNT(*)'] > 10]
    .reset_index(drop=True)
    
)
pd.testing.assert_frame_equal(Manufacturer_more_than_ten_planes, Manufacturer_more_than_ten_planes_pd) 

# Top 10 Manufacturers by Fleet Size

Description: List the top 10 manufacturers by number of planes in the fleet.

Business Question: Who are the largest contributors to our fleet by manufacturer?

In [ ]:
Top10 = pd.read_sql_query("""
    SELECT manufacturer, COUNT(*) AS howmany
    FROM planes
    GROUP BY manufacturer
    ORDER BY howmany DESC LIMIT 10    
""", conn)
Top10_pd = (
    planes.groupby('manufacturer')         
    .size()                              
    .reset_index(name='howmany')          
    .sort_values(by='howmany', ascending=False) 
    .head(10)  
    .reset_index(drop=True)
)
pd.testing.assert_frame_equal(Top10, Top10_pd) 

## Visualization: Top 10 Plane Manufacturers

A bar chart showing the top 10 manufacturers by number of planes.


In [ ]:

top10 = planes['manufacturer'].value_counts().head(10)
top10.plot(kind='bar', title='Top 10 Plane Manufacturers')
plt.ylabel('Number of Planes')
plt.show()

## Key Insights and Reflections

- Demonstrated ability to translate business questions into both SQL and pandas code.
- Validated analytical results across different technologies for reliability.
- Utilized data visualization to communicate findings effectively.
- Applied best practices in code structure, documentation, and result validation.

**Skills Demonstrated:**
- SQL (aggregation, filtering, joins, subqueries)
- Pandas (groupby, filtering, merging, reshaping)
- Data validation and testing
- Data visualization
- Clear communication of analytical process


## Conclusion

This notebook provides a comprehensive demonstration of my SQL and pandas data analysis skills, emphasizing accuracy, reproducibility, and clear communication. I am confident these skills will add value in any data-driven role.
